In [1]:
sc = SparkContext.getOrCreate()

In [2]:
df = spark.read.csv('rs.csv', inferSchema=True, header=True)

In [3]:
df.show(5)

+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
|    63|Kolya (1996)|     3|
|   226|Kolya (1996)|     5|
|   154|Kolya (1996)|     3|
|   306|Kolya (1996)|     5|
+------+------------+------+
only showing top 5 rows



In [4]:
df.count(),len(df.columns)

(100000, 3)

In [5]:
df.groupBy('rating','title').count().orderBy('count',ascending=False).orderBy('rating',ascending=False).show()

+------+--------------------+-----+
|rating|               title|count|
+------+--------------------+-----+
|     5|     Infinity (1996)|    1|
|     5| Paris, Texas (1984)|    8|
|     5|Love! Valour! Com...|    8|
|     5|  Shadow, The (1994)|    2|
|     5| If Lucy Fell (1996)|    1|
|     5|Fly Away Home (1996)|   29|
|     5|   Funny Face (1957)|    3|
|     5|Don't Be a Menace...|    2|
|     5|    Space Jam (1996)|    2|
|     5|   To Die For (1995)|   12|
|     5|Portrait of a Lad...|    2|
|     5|Cyrano de Bergera...|   19|
|     5|Leading Man, The ...|    2|
|     5|Sword in the Ston...|    6|
|     5|Total Eclipse (1995)|    1|
|     5|Reality Bites (1994)|    6|
|     5|In Love and War (...|    5|
|     5|       Mother (1996)|   17|
|     5|Raising Arizona (...|   78|
|     5|N�nette et Boni (...|    2|
+------+--------------------+-----+
only showing top 20 rows



In [6]:
df.describe().show()

+-------+------------------+--------------------+------------------+
|summary|            userId|               title|            rating|
+-------+------------------+--------------------+------------------+
|  count|            100000|              100000|            100000|
|   mean|         462.48475|                null|           3.52986|
| stddev|266.61442012750865|                null|1.1256735991443156|
|    min|                 1|'Til There Was Yo...|                 1|
|    max|               943|� k�ldum klaka (C...|                 5|
+-------+------------------+--------------------+------------------+



In [7]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



# Convert to features

In [8]:
from pyspark.ml.feature import StringIndexer

In [9]:
st = StringIndexer(inputCol='title',outputCol='title_vec').fit(df)

In [10]:
df = st.transform(df)

In [11]:
df.show(5)

+------+------------+------+---------+
|userId|       title|rating|title_vec|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
|   226|Kolya (1996)|     5|    287.0|
|   154|Kolya (1996)|     3|    287.0|
|   306|Kolya (1996)|     5|    287.0|
+------+------------+------+---------+
only showing top 5 rows



In [12]:
df.filter(df['userId'] == 1).count()

272

In [13]:
df.orderBy('userId',ascending=False).show(5)

+------+--------------------+------+---------+
|userId|               title|rating|title_vec|
+------+--------------------+------+---------+
|   943| Broken Arrow (1996)|     4|     63.0|
|   943|Right Stuff, The ...|     4|    192.0|
|   943|Legends of the Fa...|     1|    413.0|
|   943|         Jaws (1975)|     3|     49.0|
|   943|      Sabrina (1995)|     3|    128.0|
+------+--------------------+------+---------+
only showing top 5 rows



In [14]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- title_vec: double (nullable = false)



# Train test split

In [15]:
train_df, test_df = df.randomSplit([0.8,0.2])

In [16]:
train_df.count()

79917

In [17]:
test_df.count()

20083

# Train ALS model

In [18]:
from pyspark.ml.recommendation import ALS

In [19]:
rec=ALS(maxIter=10,userCol='userId',itemCol='title_vec',ratingCol='rating',nonnegative=True,coldStartStrategy="drop")

In [ ]:
rec_model=rec.fit(train_df)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64495)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ff5ac7db6e7>", line 1, in <module>
    rec_model=rec.fit(train_df)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\base.py", line 129, in fit
    return self._fit(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 321, in _fit
    java_model = self._fit_java(dataset)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 318, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "F:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspar

# Prediction and Evaluation

In [ ]:
preds = rec_model.transform(test_df)

In [ ]:
preds.show(10)